In [2]:
from __future__ import absolute_import, division, print_function

# Explore eBay API

## Imports

In [3]:
import json
import datetime
import pandas as pd
import numpy as np
from ebaysdk.exception import ConnectionError
from ebaysdk.finding import Connection
from ebaysdk.trading import Connection as Trading




## Helper functions

In [4]:
def format_time(intime):
    """
    Return datetime object formated for eBay API
    """
    out = intime.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]+'Z'
    return out

## Credentials

In [5]:
# Grab private data
if __name__ == "__main__":
    EBAY_API_APPID = os.environ['EBAY_API_APPID']
    EBAY_API_DEVID = os.environ['EBAY_API_DEVID']
    EBAY_API_CERTID = os.environ['EBAY_API_CERTID']
    EBAY_API_TOKEN = os.environ['EBAY_API_TOKEN']

## Keyword item search

In [6]:
# https://github.com/timotheus/ebaysdk-python
try:
    api = Connection(appid=APPID, config_file=None)
    response = api.execute('findItemsAdvanced', {'keywords': 'legos'})

    assert(response.reply.ack == 'Success')
    assert(type(response.reply.timestamp) == datetime.datetime)
    assert(type(response.reply.searchResult.item) == list)

    item = response.reply.searchResult.item[0]
    assert(type(item.listingInfo.endTime) == datetime.datetime)
    assert(type(response.dict()) == dict)

except ConnectionError as e:
    print(e)
    print(e.response.dict())

## Find a list of categories

In [7]:
try:    
    api = Trading(appid=APPID, devid=DEVID, certid=CERTID, token=TOKEN, config_file=None)
    response = api.execute('GetCategories', {'DetailLevel': 'ReturnAll','LevelLimit': '1'})
except ConnectionError as e:
    print(e)
    print(e.response.dict())

In [8]:
cat_array = response.dict()['CategoryArray']['Category']
category_id_dict = {}
for category in cat_array:
    if category['CategoryParentID'] != category['CategoryID']:
        print(category)
    else:
        category_id_dict[category['CategoryID']] = category['CategoryName']


In [9]:
category_id_dict

{'1': 'Collectibles',
 '10542': 'Real Estate',
 '11116': 'Coins & Paper Money',
 '11232': 'DVDs & Movies',
 '11233': 'Music',
 '11450': 'Clothing, Shoes & Accessories',
 '11700': 'Home & Garden',
 '1249': 'Video Games & Consoles',
 '12576': 'Business & Industrial',
 '1281': 'Pet Supplies',
 '1305': 'Tickets & Experiences',
 '14339': 'Crafts',
 '15032': 'Cell Phones & Accessories',
 '172008': 'Gift Cards & Coupons',
 '20081': 'Antiques',
 '220': 'Toys & Hobbies',
 '237': 'Dolls & Bears',
 '260': 'Stamps',
 '26395': 'Health & Beauty',
 '267': 'Books',
 '281': 'Jewelry & Watches',
 '293': 'Consumer Electronics',
 '2984': 'Baby',
 '316': 'Specialty Services',
 '3252': 'Travel',
 '45100': 'Entertainment Memorabilia',
 '550': 'Art',
 '58058': 'Computers/Tablets & Networking',
 '619': 'Musical Instruments & Gear',
 '625': 'Cameras & Photo',
 '64482': 'Sports Mem, Cards & Fan Shop',
 '870': 'Pottery & Glass',
 '888': 'Sporting Goods',
 '99': 'Everything Else'}

## Find all items in list of categories

In [10]:
try:
    api = Connection(appid=APPID, config_file=None)
    response = api.execute('findItemsByCategory', {'categoryId' : ['11232']},)
    assert(response.reply.ack == 'Success')
    assert(type(response.reply.timestamp) == datetime.datetime)
    assert(type(response.reply.searchResult.item) == list)

    item = response.reply.searchResult.item[0]
    assert(type(item.listingInfo.endTime) == datetime.datetime)
    assert(type(response.dict()) == dict)

except ConnectionError as e:
    print(e)
    print(e.response.dict())

## Find completed items in list of categories

In [11]:
try:
    api = Connection(appid=APPID, config_file=None)
    response = api.execute('findCompletedItems', {'categoryId' : ['11232']},)
    assert(response.reply.ack == 'Success')
    assert(type(response.reply.timestamp) == datetime.datetime)
    assert(type(response.reply.searchResult.item) == list)

    item = response.reply.searchResult.item[0]
    assert(type(item.listingInfo.endTime) == datetime.datetime)
    assert(type(response.dict()) == dict)

except ConnectionError as e:
    print(e)
    print(e.response.dict())

In [12]:
print(response.reply.searchResult.item[0])

{'itemId': '152274579098', 'isMultiVariationListing': 'true', 'topRatedListing': 'false', 'globalId': 'EBAY-US', 'title': 'Horror / Thriller Dvd Lot #1 ~ Over 90 to Choose $1.50 Each ~ $3.99 Unl Shipping', 'country': 'US', 'shippingInfo': {'expeditedShipping': 'false', 'shipToLocations': ['US', 'CA', 'GB', 'AU', 'AT', 'BE', 'FR', 'DE', 'IT', 'JP', 'ES', 'TW', 'NL', 'CN', 'HK', 'MX', 'DK', 'RO', 'SK', 'BG', 'CZ', 'FI', 'HU', 'LV', 'LT', 'MT', 'EE', 'GR', 'PT', 'CY', 'SI', 'SE', 'KR', 'ID', 'TH', 'IE', 'PL', 'RU', 'IL', 'NZ'], 'shippingServiceCost': {'_currencyId': 'USD', 'value': '3.99'}, 'oneDayShippingAvailable': 'false', 'handlingTime': '1', 'shippingType': 'Flat'}, 'galleryURL': 'http://thumbs3.ebaystatic.com/pict/1522745790984040_4.jpg', 'autoPay': 'false', 'location': 'San Manuel,AZ,USA', 'postalCode': '85631', 'returnsAccepted': 'true', 'viewItemURL': 'http://www.ebay.com/itm/Horror-Thriller-Dvd-Lot-1-Over-90-Choose-1-50-Each-3-99-Unl-Shipping-/152274579098?var=451470437158', 'se

## Create empty data frame to hold results

In [191]:
# Create empty data frame to hold results
df = pd.DataFrame(columns=('EndTimeFrom', 'EndTimeTo', 'CategoryID','items_sold','items_not_sold','items_listed'))

In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
EndTimeFrom       0 non-null object
EndTimeTo         0 non-null object
CategoryID        0 non-null object
items_sold        0 non-null object
items_not_sold    0 non-null object
items_listed      0 non-null object
dtypes: object(6)
memory usage: 0.0+ bytes


In [193]:
# Set column types
column_names = df.columns.tolist()
dtype = ['datetime64','datetime64','int64','int64','int64','int64']
column_dtypes = zip(column_names,dtype)

for k, v in column_dtypes:
    df[k] = df[k].astype(v)



In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
EndTimeFrom       0 non-null datetime64[ns]
EndTimeTo         0 non-null datetime64[ns]
CategoryID        0 non-null int64
items_sold        0 non-null int64
items_not_sold    0 non-null int64
items_listed      0 non-null int64
dtypes: datetime64[ns](2), int64(4)
memory usage: 0.0+ bytes


## Find completed items in list of categories for timeframe

In [195]:
endtimefrom = datetime.datetime(2017, 1, 24, 0, 0, 0)
endtimeto = datetime.datetime(2017, 1, 24, 1, 0, 0)

# Set API connection
api = Connection(appid=APPID, config_file=None)


# categoryId = '11232'

while endtimeto < datetime.datetime(2017, 1, 25, 0, 0, 1):
    for categoryId in category_id_dict.keys():
        # Format time string
        endtimefrom_str = format_time(endtimefrom)
        endtimeto_str = format_time(endtimeto)
        # Define query criteria - newly listed items
        para_dict = {'categoryId' : [categoryId],
                     'itemFilter': [
                        {'name': 'StartTimeFrom', 'value': endtimefrom_str},
                        {'name': 'StartTimeTo', 'value': endtimeto_str},
                        {'name': 'LocatedIn', 'value': 'US'}],
                     'paginationInput': {
                        'pageNumber': '1'},
                     }
        # Run API query - is this th right api here??
        response = api.execute('findCompletedItems', para_dict)
        # Extract numbers
        n_listed = int(response.dict()['paginationOutput']['totalEntries'])
        # Define query criteria - items sold
        para_dict = {'categoryId' : [categoryId],
                     'itemFilter': [
                        {'name': 'EndTimeFrom', 'value': endtimefrom_str},
                        {'name': 'EndTimeTo', 'value': endtimeto_str},
                        {'name': 'LocatedIn', 'value': 'US'},
                        {'name': 'SoldItemsOnly', 'value': 'True'}],
                     'paginationInput': {
                        'pageNumber': '1'},
                     }
        # Run API query
        response = api.execute('findCompletedItems', para_dict)
        # Extract numbers
        n_sold = int(response.dict()['paginationOutput']['totalEntries'])
        # Define query criteria - all items
        para_dict = {'categoryId' : [categoryId],
                     'itemFilter': [
                        {'name': 'EndTimeFrom', 'value': endtimefrom_str},
                        {'name': 'EndTimeTo', 'value': endtimeto_str},
                        {'name': 'LocatedIn', 'value': 'US'}],
                     'paginationInput': {
                        'pageNumber': '1'},
                     }
        # Run API query
        response = api.execute('findCompletedItems', para_dict)
        # Extract numbers
        n_all = int(response.dict()['paginationOutput']['totalEntries'])
        # Calculate items not sold
        n_not_sold = n_all - n_sold
        print(categoryId,' : ',endtimefrom, '--', endtimeto,' : ', n_all)
        # Add result to data frame
        df.loc[df.shape[0]]= [endtimefrom,endtimeto,int(categoryId),n_sold,n_not_sold,n_listed]
    # Indent time
    endtimefrom = endtimefrom + datetime.timedelta(hours=1.0)
    endtimeto = endtimeto + datetime.timedelta(hours=1.0)

1305  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  68
625  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  1469
888  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  13330
550  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  1482
58058  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  7096
172008  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  160
237  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  2326
1  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  35278
11116  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  3782
3252  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  286
870  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  3413
2984  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  753
26395  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  11185
281  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  19695
64482  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  14346
220  :  2017-01-24 00:00:00 -- 2017-01-24 01:00:00  :  14014
11700  :  2017-01-24 00:00:

KeyboardInterrupt: 

In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816 entries, 0 to 815
Data columns (total 6 columns):
EndTimeFrom       816 non-null datetime64[ns]
EndTimeTo         816 non-null datetime64[ns]
CategoryID        816 non-null int64
items_sold        816 non-null int64
items_not_sold    816 non-null int64
items_listed      816 non-null int64
dtypes: datetime64[ns](2), int64(4)
memory usage: 44.6 KB


## Save data frame to disk

In [190]:
# Remember to rename the temporary file!
df.to_csv('tmp.csv', sep=',')

In [174]:
df

,EndTimeFrom,EndTimeTo,CategoryID,items_sold,items_not_sold,items_listed
0,2017-01-23 00:00:00,2017-01-23 01:00:00,1305,11,120,18
1,2017-01-23 00:00:00,2017-01-23 01:00:00,625,565,1218,167
2,2017-01-23 00:00:00,2017-01-23 01:00:00,888,3664,11291,763
3,2017-01-23 00:00:00,2017-01-23 01:00:00,550,284,2309,53
4,2017-01-23 00:00:00,2017-01-23 01:00:00,58058,1321,3823,550
5,2017-01-23 00:00:00,2017-01-23 01:00:00,172008,96,53,56
6,2017-01-23 00:00:00,2017-01-23 01:00:00,237,951,2676,157
7,2017-01-23 00:00:00,2017-01-23 01:00:00,1,10769,40190,1233
8,2017-01-23 00:00:00,2017-01-23 01:00:00,11116,2078,4865,224
9,2017-01-23 00:00:00,2017-01-23 01:00:00,3252,59,256,15


In [196]:
# Remove some entries
# idx = df[df['EndTimeTo'] == endtimeto].index.values.tolist()
# print(idx)

In [197]:
# df.drop(df.index[648])

# Notes

"Looks like EndedWithoutSales is only supported for 14 days after which they are removed from the listings and only sold items show."
https://forums.developer.ebay.com/questions/5594/findcompleteditems-and-sellingstatus.html